In [80]:
import numpy as np
import sympy as sp

In [81]:
pi = 4.0 * np.arctan(1.0)
planck = 6.62606957e-34         # J s
avogadro = 6.0221413e23
kb = 1.3806488e-23              # J K^-1
speed_of_light = 299792458.0    # m s^-1
amu = 1.660538921e-27           # kg
gas_constant = avogadro*kb
hbar = planck/(2.0*pi)
global_eps = 1.0e-10

# Calculation of thermodynamic data

This is done in the end of all qce iterations. </br>
The partition functions, volumes and populations are calculated for each molecule at each temperature. </br>
They are then used for the calculation of the thermodynamic data.

## calculation of the system partition function
Attention: This subroutine is missing the summand arising from the particle </br>
indistinguishability.

$$ Q_{\text{sys}}(T) = \prod_{i=1}^{N_{\text{clust}}} Q(T)^{N_{\text{clust}}(T)} $$
$$ \ln{Q_{\text{sys}}(T)} = \sum_{i=1}^{N_{\text{clust}}} N_{\text{clust}}(T) \ln{Q(T)} $$

In [82]:
def calc_lnq_sys(ntemp, nclust, pop, ln_clust, lnq_sys):
    # ntemp     : input     - number of temperatures at which the system is simulated
    # nclust    : input     - number of clusters in the system
    # pop       : input     - population of each cluster at each temperature
    # ln_clust  : input     - ln of the population of each cluster
    # lnq_sys   : output    - ln of the partition function of the system
    
    lnq_sys = np.zeros(ntemp)
    for i in range(ntemp):
        for j in range(nclust):
            lnq_sys[i] += pop[i][j] * ln_clust[i][j]
            
    return lnq_sys

#### Tests

In [83]:
ntemp = 5
nclust = 4

pop = np.array([[0.5, 0.3, 0.1, 0.1],
                [0.4, 0.3, 0.2, 0.1],
                [0.3, 0.3, 0.3, 0.1],
                [0.2, 0.3, 0.4, 0.1],
                [0.1, 0.3, 0.5, 0.1]])

ln_clust = np.array([[-5.2, 3.8, 2.3, 1.3],
                     [-4.2, 3.3, 2.8, 1.3],
                     [-3.2, 3.3, 3.3, 1.3],
                     [-2.2, 3.3, 4.3, 1.3],
                     [-1.2, 3.3, 5.3, 1.3]])

lnq_sys = np.zeros(ntemp)

lnq_sys = calc_lnq_sys(ntemp, nclust, pop, ln_clust, lnq_sys)
print(lnq_sys)

[-1.10000000e+00 -3.33066907e-16  1.15000000e+00  2.40000000e+00
  3.65000000e+00]


## Add the particle indistinguishability 

$$ Q_{\text{sys}}(T) = \prod_{i=1}^{N_{\text{clust}}} \frac{1}{N!}Q(T)^{N_{\text{clust}}(T)} $$
$$ \ln{Q_{\text{sys}}(T)} = \sum_{i=1}^{N_{\text{clust}}} N_{\text{clust}}(T) \ln{Q(T)} - \sum_{i=1}^{N_{\text{clust}}} \ln{N!}$$

In [84]:
def add_lnq_indi(ntemp, nclust, pop, lnq_sys):
    # ntemp     : input     - number of temperatures at which the system is simulated
    # nclust    : input     - number of clusters in the system
    # pop       : input     - population of each cluster at each temperature
    # lnq_sys   : input     - ln of the partition function of the system
    
    lnq_indi = np.zeros(ntemp)
    for i in range(ntemp):
        for j in range(nclust):
            lnq_sys[i] += - sp.log(sp.factorial(pop[i][j]))
            
    return lnq_indi

#### Tests

In [85]:
ntemp = 5
nclust = 4

pop = np.array([[0.5, 0.3, 0.1, 0.1],
                [0.4, 0.3, 0.2, 0.1],
                [0.3, 0.3, 0.3, 0.1],
                [0.2, 0.3, 0.4, 0.1],
                [0.1, 0.3, 0.5, 0.1]])

lnq_sys = [-1.1, -3.33066907e-16,  1.15, 2.4,  3.65]

add_lnq_indi(ntemp, nclust, pop, lnq_sys)
print(lnq_sys)

[-0.771298070337215, 0.363034254943387, 1.52439686978342, 2.76303425494339, 3.97870192966279]


## Calculation of the Helmholtz energy
$$ A(T) = -kT \ln{Q} $$

In [86]:
def calculate_helmholtz_energy(ntemp, temp, lnq):
    # ntemp     : input     - number of temperatures at which the system is simulated
    # temp      : input     - temperature of the system
    # lnq       : input     - ln of the partition function of the system
    # A         : output    - Helmholtz energy of the system
    
    A = np.zeros(ntemp)
    for i in range(ntemp):
        A[i] = - kb*temp[i] * lnq[i]
        
    return A

#### Tests

In [87]:
ntemp = 8
temp = np.array([100.15, 200.0, 300.0, 406.05, 500.0, 604.0, 730.4, 850.0])
lnq = np.array([100.03, 2.45, 3.2, 4.99, 5.04, 6.0, 7.089, 8.54])

A = calculate_helmholtz_energy(ntemp, temp, lnq)
print(A)

[-1.38313459e-19 -6.76517912e-21 -1.32542285e-20 -2.79745610e-20
 -3.47923498e-20 -5.00347125e-20 -7.14873109e-20 -1.00221296e-19]
